In [1]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [5]:
client.create_experiment(name="my-cool-experiment")

'3'

In [10]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"] 
)

In [11]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 1fec443a8d8f481095d7edfbdec84e70, rmse: 6.3184
run id: 196c2a346deb4ba9b425ae924c5fffcf, rmse: 6.3184
run id: afc5d01724734a56b2298131432b5ae8, rmse: 12.2126


In [12]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [20]:
run_id = "196c2a346deb4ba9b425ae924c5fffcf"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-model")

Registered model 'nyc-taxi-model' already exists. Creating a new version of this model...
Created version '3' of model 'nyc-taxi-model'.


<ModelVersion: aliases=[], creation_timestamp=1732904845715, current_stage='None', description=None, last_updated_timestamp=1732904845715, name='nyc-taxi-model', run_id='196c2a346deb4ba9b425ae924c5fffcf', run_link=None, source='/home/rajan/mlops-camp/02-experiment-tracking/mlruns/1/196c2a346deb4ba9b425ae924c5fffcf/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [24]:
model_name = "nyc-taxi-model"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 3, stage: None


/tmp/ipykernel_5836/29955185.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [28]:
client.transition_model_version_stage(
    name=model_name,
    version=3,
    stage="staging",
    archive_existing_versions=False
)

/tmp/ipykernel_5836/3059825122.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1732904845715, current_stage='Staging', description=None, last_updated_timestamp=1732904990171, name='nyc-taxi-model', run_id='196c2a346deb4ba9b425ae924c5fffcf', run_link=None, source='/home/rajan/mlops-camp/02-experiment-tracking/mlruns/1/196c2a346deb4ba9b425ae924c5fffcf/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [106]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(stage, X_test, y_test, name="nyc-taxi-model"):
    # Ensure we're using the same SQLite tracking URI
    mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
    
    # Load the model directly using the run_id
    model_path = f"runs:/{run_id}/models_mlflow"
    print(f"Loading model from: {model_path}")
    
    model = mlflow.pyfunc.load_model(model_path)
    y_pred = model.predict(X_test)
    
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}


In [107]:
df = read_dataframe("data/green_tripdata_2021-03.parquet")

In [108]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/home/rajan/mlops-camp/02-experiment-tracking/preprocessor'

In [109]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [110]:
X_test = preprocess(df, dv)

In [111]:
target = "duration"
y_test = df[target].values

In [112]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

Loading model from: runs:/196c2a346deb4ba9b425ae924c5fffcf/models_mlflow
CPU times: user 18.8 s, sys: 110 ms, total: 18.9 s
Wall time: 2.83 s


/home/rajan/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'rmse': np.float64(6.2702965482607915)}

In [113]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

Loading model from: runs:/196c2a346deb4ba9b425ae924c5fffcf/models_mlflow
CPU times: user 18 s, sys: 11.9 ms, total: 18 s
Wall time: 2.33 s


/home/rajan/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'rmse': np.float64(6.2702965482607915)}

In [114]:
client.transition_model_version_stage(
    name=model_name,
    version=3,
    stage="Production",
    archive_existing_versions=True
)

/tmp/ipykernel_5836/684772386.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1732904845715, current_stage='Production', description=None, last_updated_timestamp=1732906770228, name='nyc-taxi-model', run_id='196c2a346deb4ba9b425ae924c5fffcf', run_link=None, source='/home/rajan/mlops-camp/02-experiment-tracking/mlruns/1/196c2a346deb4ba9b425ae924c5fffcf/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>